### Hints for homework with the commands2 library - 2021 1231 CJH
---

### What is already in the template
* The robotpy example (hatch) has a working drivetrain (drivesusbsystem.py) and a drivetrain sim
* It has a robotcontainer.py that can show you how to 
  * import and instantiate subsystems
  * bind commands to joystick buttons
* The template already has several commands written in the comands folder
  * look through them and understand what they do
  * use these as templates to help you make your own
* Common mistakes
  * not getting an intial value or resetting the encoder / gyro before the action if you are tracking
  * not knowing what the encoders are reporting as it moves - use the sim to see what is changing as you move
  * look at other examples for ideas (the ramsete example has the right setup for encoder units)
  

---
### #1: Driving for a set amount of time

General hints:
* make a new command.  the easiest way is to copy one of the simple ones and rename it
* you can use the command drivedistance.py for help
  * that one drives a distance, so it is checking the encoder to see if it should end
  * if you just want to drive for a time, you can set the command timeout and check for timed out in the isFinished() part of the command
  * they took away the isTimedOut and setTimeOut functions in commands2
  * so you should get the time when you initialize and see if the difference is greater than the time you wanted to drive
 

In [20]:
import wpilib
import commands2

In [21]:
drive_time = 2 # for instance, assign to self.drive time in init

# in initialize stage of command
start_time = wpilib.Timer.getMatchTime()

# in the isFinished stage of the command
elapsed_time = wpilib.Timer.getMatchTime() - start_time
def isFinished():
    if elapsed_time > drive_time:
        return True
    else:
        return False

---
### #2: Driving for a set distance with a PID

General hints:
* make a new command.  the easiest way is to copy one of the simple ones and rename it
* you can use the command drivedistance.py for help
  * that one drives a distance, so it is checking the encoder to see if it should end
  *  let's not do that.  just let it run until the button is let go so you can see it **overshoot** and recover
  * but instead of just running a fixed speed to `self.robot.drivetrain.arcade_drive(1,0)`, you need to use a PID.  You could write a custom one, like in the 2021 autonomous_drive_pid, or use the wpilib's built in one 
  * you probably want to **clamp**  the output between -1 and 1 - although the max output of the drive will do that for you - it's still good practice

In [22]:
from wpilib.controller import PIDController

In [23]:
# in initialize
setpoint = 2 # or whatever, pass to init and assign to self.setpoint
kp = 0.5
ki = 0.0
kd = 0.0
test_controller = PIDController(Kp=kp, Ki=ki, Kd=kd)
test_controller.setSetpoint(setpoint)

# in execute now you calculate the PID value
measured_distance = 0.5 # self.drive.getAverageEncoderDistance()
output = test_controller.calculate(measured_distance)
#self.drive.arcadeDrive(output, 0)  # drives straight

# in isFinished() you probably just want to 

---
### #3: Rotating for a set angle with a PID

* it's the same as #2 for the most part
* you now need to use the gyro instead of the wheel encoder
* but there is no gyro in the template_auto project
  * look in the ramsete project for how to add a gyro and a get_heading() function 
  
